In [1]:
#加工前データ
import xlrd
import pprint
import openpyxl
import numpy as np
from datetime import datetime as dt
import pandas as pd
from sklearn import preprocessing

In [2]:
bed_data_filename = 'bed_data.csv'
before_csv = 'osaka_data_before.csv'
data_filename = 'osaka_data_before.xlsx'
save_filename = 'before.xlsx'
data_sheet = 'Sheet1'

In [3]:
#開始日と終了日を参照する
df = pd.read_csv('./data/'+bed_data_filename, encoding='shift-jis', index_col=0, parse_dates=['日付'])
start_day = df.index[0]
end_day = df.index[-1]
start_day, end_day

(Timestamp('2020-07-30 00:00:00'), Timestamp('2021-09-18 00:00:00'))

In [4]:
test_start = dt(2020,10,1,0)
#test_end = dt(2021,1,11,0)
day_num = (end_day-start_day).days
total_data_num = day_num+1
test_data_num = (end_day-test_start).days+1
teach_data_num = total_data_num-test_data_num

#メンバシップ関数の数
func_num = 3
#入力変数
x_dimension = 6
#学習する変数の数
total_dimension = func_num**x_dimension
#学習回数
max_iter = 100
omega_iter = 100
#学習係数
l_rate = 0.5

In [5]:
# csvファイルを読み込む
before_csv = pd.read_csv('./data/'+before_csv, encoding='shift-jis')

# エクセルファイルに書き込む
writer = pd.ExcelWriter('./data/'+data_filename)
before_csv.to_excel(writer,data_sheet,index=False) # header=False

#　保存
writer.save()

In [6]:
#データ関連関数
def get_value_list(t_2d):
    return([[cell.value for cell in row] for row in t_2d])

def get_list_2d(sheet, start_row, end_row, start_col, end_col):
    return get_value_list(sheet.iter_rows(min_row=start_row,
                                          max_row=end_row,
                                          min_col=start_col,
                                         max_col=end_col))

def write_list_2d(sheet, l_2d, start_row, start_col):
    for y, row in enumerate(l_2d):
        for x, cell in enumerate(row):
            sheet.cell(row=start_row + y,
                       column=start_col + x,
                       value=l_2d[y][x])

In [7]:
#μやωからyを計算
def cal_y(m, total_m, y, w, omega):
    temp = 0
    for (i, j, l) in zip(m, w, omega):
        temp += i * j * l
    y[1] = temp/total_m

In [8]:
#データを扱いやすく変形
def set_data(y, Y, data, b, c, omega):
    
    for i in range(total_data_num):
        y.append([(Y[i][0]-np.min(Y))/(np.max(Y)-np.min(Y)),(np.mean(Y)-np.min(Y))/(np.max(Y)-np.min(Y))])
        
    for i in range(total_dimension):
        omega[i] = (i+1)/total_dimension

In [9]:
#メンバーシップ関数
def fun(c, b, fun_data, data):    
    for i in range(total_data_num):
        for j in range(x_dimension):
            for k in range(func_num):
                #print(i,j,data[i][j])
                if data[i][j] < c[k]-b[k]:
                    fun_data[i][k][j] = 0
                if data[i][j] > c[k]+b[k]:
                    fun_data[i][k][j] = 0
                if data[i][j] >= c[k]-b[k] and data[i][j] <= c[k]:
                    fun_data[i][k][j] = 1-abs(c[k]-data[i][j])*(1/b[k])
                if data[i][j] > c[k] and data[i][j] <= c[k]+b[k]:
                    fun_data[i][k][j] = 1-abs(c[k]-data[i][j])*(1/b[k])

In [10]:
#データに関数を適用する関数
def rule(m, total_m, fun_data, data, c, b, w):
    fun(c, b, fun_data, data)
    
    for d_num in range(total_data_num):
        for j in range(x_dimension):
            times = 0
            R = func_num**(x_dimension-1-j)
            for p in range(R):
                for i in range(func_num):
                    r = func_num**j
                    for k in range(r):
                        m[d_num][times] *= fun_data[d_num][i][j]
                        times+=1
                        
    #学習用とテスト用のμの総和
    for j in range(total_data_num):
        for i in range(total_dimension):
            total_m[j] += m[j][i] * w[i]

"""
for i in range(omega_iter):    
    for j in range(teach_data_num):
        for dim in range(total_dimension):
            omega[dim] -= l_rate * (-y[j][0]+y[j][1]) * m[j][dim] / total_m[j]
            w[dim] = w[dim] - l_rate *  (-y[j][0]+y[j][1]) * m[j][dim] * (omega[dim]-y[j][1]) / total_m[j]"""
            '''if temp_w < 0:
                w[dim] = 0
            else:
                w[dim] = temp_w'''

In [11]:
#最急降下法 改良
def gradient_descent(total_m, m, y, w, omega, mse):
    temp_w = 0
    
    #後件部の学習
    for i in range(omega_iter):
        for j in range(teach_data_num):
            for dim in range(total_dimension):
                omega[dim] -= l_rate * (-y[j][0]+y[j][1]) * m[j][dim] / total_m[j]
                w[dim] = w[dim] - l_rate *  (-y[j][0]+y[j][1]) * m[j][dim] * (omega[dim]-y[j][1]) / total_m[j]
            temp1 = 0
            temp2 = 0
            for r, W, ome in zip(m[j], w, omega):
                temp1 += r * W * ome
                temp2 += r * W
            total_m[j] = temp2
            y[j][1] = temp1/total_m[j]
        #MES(y, mse, i, w, omega, 'teach')
        if i%10 == 0:
            print(str(i)+"回目")
            #pprint.pprint(y)
            """for r, W, ome in zip(m[j], w, omega):
                temp1 += r * W * ome
                temp2 += r * W
            total_m[j] = temp2
            y[j][1] = temp1/total_m[j]
        #print(y[0][1])
        MES(y, mse, i, w, omega, 'teach')
        if i%10 == 0:
            print(str(i)+"回目")"""
    #重みの学習結果を残しておく
    #時間短縮のため
    save_result(y, w, omega, m, total_m, 1)
    
    print("後件部学習終了")
    print("重み学習終了")
    
    #testデータで
    for i in range(teach_data_num,total_data_num):
        temp = 0
        for l,r in zip(m[i],w):
            temp += l*r
            """if r > 0.75:
                temp += l*r"""
        total_m[i] = temp
        
        #エラー処理
        if total_m[i] == 0:
            print(CV_num,i)

        temp = 0
        for r, W, ome in zip(m[i], w, omega):
            temp += r * W * ome
            """if W > 0.75:
                temp += r * W * ome"""
        y[i][1] = temp/total_m[i]

    #MES(y, mse, max_iter, w, omega, 'test')
    save_result(y, w, omega, m, total_m, 2)

In [12]:
def MES(y, mse, n, w, omega, litera):
    temp = 0
    
    if litera=='teach':
        for i in range(teach_data_num):
            temp += (y[i][1] - y[i][0])**2
        mse = temp/teach_data_num
    else:
        for i in range(teach_data_num,total_data_num):
            temp += (y[i][1] - y[i][0])**2
        mse = temp/test_data_num
    
    wc = openpyxl.load_workbook('./output/osaka/'+save_filename, data_only=True)
    for sheet_name in wc.sheetnames:
        if sheet_name == 'output':
            wc["sq_error"].cell(column=1, row=1+n, value = mse)
            
            wc.save('./output/osaka/'+save_filename) 
        else:
            err_sheet = wc.create_sheet('sq_error')
            err_sheet.cell(column=1, row=1+n, value = mse)
    return 

In [13]:
#excelの保存
def save_result(output_list, w, omega, m, total_m, num):
    wc = openpyxl.load_workbook('./output/osaka/'+save_filename, data_only=True)
    for sheet_name in wc.sheetnames:
        if sheet_name == 'output':
            if num == 1:
                omega_sheet = wc["omega"]
                for i in range(total_dimension):
                    omega_sheet.cell(column=1, row=i+1, value = omega[i])

                w_sheet = wc["w"]
                for i in range(total_dimension):
                    w_sheet.cell(column=1, row=i+1, value = w[i])

                m_sheet = wc["m"]
                for i in range(total_data_num):
                    for j in range(total_dimension):
                        m_sheet.cell(column=1+i, row=1+j, value = m[i][j])

                total_m_sheet = wc["total_m"]
                for i in range(total_data_num):
                    total_m_sheet.cell(column=1+i, row=1, value = total_m[i])
            else:
            
                write_list_2d(wc["output"],[['理想値','出力値']], 1, 1)
                write_list_2d(wc["output"],output_list, 2, 1)

                #wc["sq_error"].cell(column=1, row=1, value = mse)

            wc.save('./output/osaka/'+save_filename)
            print("Already exist. Save complited ")  
            return 
        
    '''output_sheet = wc.create_sheet('output')
    write_list_2d(output_sheet,[['理想値','出力値']], 1, 1)
    write_list_2d(output_sheet,output_list, 2, 1)
    #err_sheet = wc.create_sheet('sq_error')
    #err_sheet.cell(column=1, row=1, value = mse)
    
    w_sheet = wc.create_sheet('w')
    for i in range(total_dimension):
        w_sheet.cell(column=1, row=i+1, value = w[i])
        
    w_sheet = wc.create_sheet('omega')
    for i in range(total_dimension):
        w_sheet.cell(column=1, row=i+1, value = omega[i])
    
    wc.save('./output/osaka/'+save_filename)
    print("Save complited ")
    return '''

In [14]:
#excelの保存
def save_fix_result(fix):
    wc = openpyxl.load_workbook('./output/osaka/'+save_filename, data_only=True)
    for sheet_name in wc.sheetnames:
        if sheet_name == 'output':
            fix_sheet = wc["fix"]
            write_list_2d(fix_sheet,[['理想値','出力値']], 1, 1)
            write_list_2d(fix_sheet,fix, 2, 1)
    wc.save('./output/osaka/'+save_filename)
    print("Already exist. Save complited ")  
    return 

In [15]:
#出力yを導出する関数
def make_output(total_m, m, y, fun_data, data, omega, c, b, w, sq_err):
    
    print("学習開始")
    gradient_descent(total_m, m, y, w, omega, sq_err)
    print("学習完了")
    
    return 0

In [23]:
def main():
    #Excelの指定
    wb = openpyxl.load_workbook('./data/'+data_filename, data_only=True)
    data_sheet = wb['Sheet1']

    '''
    data：データ
    y：病床占有率
    fun_data：関数を通したデータの値
    m：ルールに対する適合度
    total_m：適合度の総和
    omega：ルール後件部
    w：ルールの重み
    mse：平均2乗誤差
    c：関数の中心
    b：関数の幅
    '''
    
    start_row = 2
    data = get_list_2d(data_sheet, start_row, total_data_num+start_row-1, 2, x_dimension+1)
    Y = get_list_2d(data_sheet, start_row, total_data_num+start_row-1, x_dimension+2, x_dimension+2)
    y = []
    fun_data = np.empty((total_data_num, func_num, x_dimension))
    m = np.ones((total_data_num, total_dimension))
    total_m = np.zeros(total_data_num)
    omega = np.zeros(total_dimension)
    w = np.ones(total_dimension)
    mse = np.zeros(max_iter)
    
    #中心と幅の初期値の設定
    c = [0, 0.5, 1] 
    b = [1, 0.5, 1] #変えるべき
    
    #参照問題でここで正規化
    #保存して、読み込みでもいいかも
    mm = preprocessing.MinMaxScaler()
    mm_data = mm.fit_transform(data)
    Y = mm.fit_transform(Y)
    print("正規化完了")
    
    #pprint.pprint(Y)
    
    #データのセット
    set_data(y, Y, mm_data, b, c, omega)   
    print("データセット完了")
    rule(m, total_m, fun_data, mm_data, c, b, w)
    
    make_output(total_m, m, y, fun_data, mm_data, omega, c, b, w, mse)
    
    #予測値を%表記に逆標準化
    #結合・逆変換のために型変換
    y_ = np.array(y)
    Y_ = np.array(Y)
    mm_y_ = y_[:, 1].reshape(-1,1)
    mm_y = mm.inverse_transform(mm_y_)
    fix_y = np.append(mm_y, Y_, axis=1)
    #pprint.pprint(fix_y)
    
    save_fix_result(fix_y)
    
if __name__ == '__main__':
    
    main()
    

正規化完了
データセット完了
学習開始
0回目
[[0.11688311688311692, 0.0420319520761419],
 [0.1428571428571429, 0.02596526247373216],
 [0.15584415584415584, -0.02727032633764781],
 [0.2207792207792208, -0.06941362474520466],
 [0.2337662337662338, -0.17875913424051668],
 [0.2337662337662338, -0.11078478392747115],
 [0.2207792207792208, -0.15582903945605794],
 [0.24675324675324678, -0.11142818881275968],
 [0.24675324675324678, -0.13935621039689533],
 [0.28571428571428575, -0.22212618567654427],
 [0.31168831168831174, -0.24587077267229757],
 [0.3376623376623377, -0.36531196087710405],
 [0.3376623376623377, -0.36590052039932225],
 [0.3506493506493507, -0.2680151955578692],
 [0.3506493506493507, -0.253027565321656],
 [0.3506493506493507, -0.21547681647832137],
 [0.3896103896103897, -0.3633529658730306],
 [0.41558441558441567, -0.3885401877264066],
 [0.4025974025974027, -0.38795144208960103],
 [0.42857142857142866, -0.23292423378446825],
 [0.44155844155844165, -0.21795992066253925],
 [0.44155844155844165, -0.3811

 [0.5194805194805194, 0.4379995004995006],
 [0.5454545454545454, 0.4379995004995006],
 [0.5714285714285714, 0.4379995004995006],
 [0.6233766233766234, 0.4379995004995006],
 [0.6623376623376624, 0.4379995004995006],
 [0.6883116883116884, 0.4379995004995006],
 [0.7142857142857144, 0.4379995004995006],
 [0.7142857142857144, 0.4379995004995006],
 [0.7532467532467533, 0.4379995004995006],
 [0.7792207792207793, 0.4379995004995006],
 [0.8181818181818183, 0.4379995004995006],
 [0.8831168831168832, 0.4379995004995006],
 [0.8961038961038961, 0.4379995004995006],
 [0.8961038961038961, 0.4379995004995006],
 [0.9090909090909092, 0.4379995004995006],
 [0.9090909090909092, 0.4379995004995006],
 [0.9350649350649352, 0.4379995004995006],
 [0.9610389610389611, 0.4379995004995006],
 [1.0, 0.4379995004995006],
 [0.8961038961038961, 0.4379995004995006],
 [0.8961038961038961, 0.4379995004995006],
 [0.9090909090909092, 0.4379995004995006],
 [0.922077922077922, 0.4379995004995006],
 [0.9090909090909092, 0.437

20回目
[[0.11688311688311692, 0.2767670389609869],
 [0.1428571428571429, 0.2747840894857312],
 [0.15584415584415584, 0.27458320603787434],
 [0.2207792207792208, 0.27692176096286697],
 [0.2337662337662338, 0.27620059719554296],
 [0.2337662337662338, 0.27286939185530595],
 [0.2207792207792208, 0.27323555519935505],
 [0.24675324675324678, 0.27100426444988407],
 [0.24675324675324678, 0.27088706947116],
 [0.28571428571428575, 0.2737960701697813],
 [0.31168831168831174, 0.27423481255338267],
 [0.3376623376623377, 0.27621163594399567],
 [0.3376623376623377, 0.273855414469355],
 [0.3506493506493507, 0.27177517449772826],
 [0.3506493506493507, 0.2721893680508188],
 [0.3506493506493507, 0.27017244866929],
 [0.3896103896103897, 0.27542336801774603],
 [0.41558441558441567, 0.27725490056280977],
 [0.4025974025974027, 0.2781514776314217],
 [0.42857142857142866, 0.27281254006489464],
 [0.44155844155844165, 0.2728343581800235],
 [0.44155844155844165, 0.2782977872729803],
 [0.44155844155844165, 0.2760212

30回目
[[0.11688311688311692, 0.27674831253672383],
 [0.1428571428571429, 0.27480615299446476],
 [0.15584415584415584, 0.27458472059205313],
 [0.2207792207792208, 0.2768877734944883],
 [0.2337662337662338, 0.2761456339263869],
 [0.2337662337662338, 0.27290065297461974],
 [0.2207792207792208, 0.27324940925052343],
 [0.24675324675324678, 0.27108697653718694],
 [0.24675324675324678, 0.2709658233951786],
 [0.28571428571428575, 0.273807616620929],
 [0.31168831168831174, 0.274246564926963],
 [0.3376623376623377, 0.27616379036886457],
 [0.3376623376623377, 0.2738486756594114],
 [0.3506493506493507, 0.27183619811377946],
 [0.3506493506493507, 0.2722773354922292],
 [0.3506493506493507, 0.27032490450225005],
 [0.3896103896103897, 0.27543037769918904],
 [0.41558441558441567, 0.2772420739188792],
 [0.4025974025974027, 0.27818504062237254],
 [0.42857142857142866, 0.27297178248442117],
 [0.44155844155844165, 0.2729998846951791],
 [0.44155844155844165, 0.27831696962663754],
 [0.44155844155844165, 0.276

40回目
[[0.11688311688311692, 0.27674390188769354],
 [0.1428571428571429, 0.2748421356830364],
 [0.15584415584415584, 0.2746001570193416],
 [0.2207792207792208, 0.27686782502928103],
 [0.2337662337662338, 0.27610487958797925],
 [0.2337662337662338, 0.27294529060081674],
 [0.2207792207792208, 0.2732766390241345],
 [0.24675324675324678, 0.27118250392184956],
 [0.24675324675324678, 0.2710572729898634],
 [0.28571428571428575, 0.27383219128759206],
 [0.31168831168831174, 0.27427124928171265],
 [0.3376623376623377, 0.276129219856416],
 [0.3376623376623377, 0.27385484407841765],
 [0.3506493506493507, 0.2719094459259631],
 [0.3506493506493507, 0.2723773155838456],
 [0.3506493506493507, 0.27048895838166187],
 [0.3896103896103897, 0.27544962726470973],
 [0.41558441558441567, 0.27724149134268494],
 [0.4025974025974027, 0.2782306916682518],
 [0.42857142857142866, 0.27314211209892775],
 [0.44155844155844165, 0.2731763141792694],
 [0.44155844155844165, 0.27834776064172806],
 [0.44155844155844165, 0.27

...

60回目
[[0.11688311688311692, 0.27673506807337755],
 [0.1428571428571429, 0.2749137644479523],
 [0.15584415584415584, 0.2746308808105933],
 [0.2207792207792208, 0.2768281170222206],
 [0.2337662337662338, 0.2760238242353582],
 [0.2337662337662338, 0.2730342250349326],
 [0.2207792207792208, 0.27333097620672897],
 [0.24675324675324678, 0.2713729455375455],
 [0.24675324675324678, 0.2712394801776713],
 [0.28571428571428575, 0.2738812784072641],
 [0.31168831168831174, 0.27432055507893793],
 [0.3376623376623377, 0.2760606233668214],
 [0.3376623376623377, 0.27386734330209117],
 [0.3506493506493507, 0.27205543381993896],
 [0.3506493506493507, 0.27257660132941813],
 [0.3506493506493507, 0.2708159768443608],
 [0.3896103896103897, 0.27548812830729447],
 [0.41558441558441567, 0.2772404950123691],
 [0.4025974025974027, 0.27832193315240195],
 [0.42857142857142866, 0.27348155160627063],
 [0.44155844155844165, 0.2735279125973506],
 [0.44155844155844165, 0.2784092129342289],
 [0.44155844155844165, 0.27662

70回目
[[0.11688311688311692, 0.2767306191970538],
 [0.1428571428571429, 0.27494938712342204],
 [0.15584415584415584, 0.2746461439216344],
 [0.2207792207792208, 0.27680833240990005],
 [0.2337662337662338, 0.27598349815877177],
 [0.2337662337662338, 0.27307850005892653],
 [0.2207792207792208, 0.2733580610666129],
 [0.24675324675324678, 0.27146784030631],
 [0.24675324675324678, 0.2713302189645532],
 [0.28571428571428575, 0.27390576910646575],
 [0.31168831168831174, 0.27434515558031447],
 [0.3376623376623377, 0.27602657414462844],
 [0.3376623376623377, 0.27387365284539955],
 [0.3506493506493507, 0.2721281551052929],
 [0.3506493506493507, 0.2726758904041294],
 [0.3506493506493507, 0.2709789282789207],
 [0.3896103896103897, 0.2755073593559271],
 [0.41558441558441567, 0.27724006035794335],
 [0.4025974025974027, 0.27836750718830294],
 [0.42857142857142866, 0.27365064980961734],
 [0.44155844155844165, 0.27370306967928826],
 [0.44155844155844165, 0.27843985564094803],
 [0.44155844155844165, 0.276

80回目
[[0.11688311688311692, 0.2767261481617558],
 [0.1428571428571429, 0.2749848823466882],
 [0.15584415584415584, 0.2746613409020459],
 [0.2207792207792208, 0.27678859325085603],
 [0.2337662337662338, 0.27594330577053333],
 [0.2337662337662338, 0.2731226477671857],
 [0.2207792207792208, 0.2733850903340725],
 [0.24675324675324678, 0.2715625195337874],
 [0.24675324675324678, 0.2714207166895798],
 [0.28571428571428575, 0.2739302249065014],
 [0.31168831168831174, 0.2743697219178488],
 [0.3376623376623377, 0.2759926903462217],
 [0.3376623376623377, 0.27388000278690205],
 [0.3506493506493507, 0.2722006961684912],
 [0.3506493506493507, 0.2727749464500162],
 [0.3506493506493507, 0.27114151263553365],
 [0.3896103896103897, 0.27552657772752676],
 [0.41558441558441567, 0.27723966797926586],
 [0.4025974025974027, 0.2784130528906724],
 [0.42857142857142866, 0.2738193388658854],
 [0.44155844155844165, 0.2738778036000767],
 [0.44155844155844165, 0.2784704435884521],
 [0.44155844155844165, 0.27693749

90回目
[[0.11688311688311692, 0.2767216543526744],
 [0.1428571428571429, 0.2750202503907813],
 [0.15584415584415584, 0.2746764715622755],
 [0.2207792207792208, 0.2767688989609724],
 [0.2337662337662338, 0.27590324655681264],
 [0.2337662337662338, 0.2731666687758675],
 [0.2207792207792208, 0.27341206419429565],
 [0.24675324675324678, 0.27165698458300724],
 [0.24675324675324678, 0.2715109749541659],
 [0.28571428571428575, 0.27395464607711006],
 [0.31168831168831174, 0.2743942546759848],
 [0.3376623376623377, 0.27595897155085475],
 [0.3376623376623377, 0.2738863934708558],
 [0.3506493506493507, 0.2722730581747179],
 [0.3506493506493507, 0.27287377158202736],
 [0.3506493506493507, 0.2713037334728416],
 [0.3896103896103897, 0.2755457836643269],
 [0.41558441558441567, 0.27723931784440714],
 [0.4025974025974027, 0.2784585723471998],
 [0.42857142857142866, 0.2739876226518649],
 [0.44155844155844165, 0.2740521180190461],
 [0.44155844155844165, 0.27850097743374175],
 [0.44155844155844165, 0.277090

後件部学習終了
重み学習終了
学習完了
Already exist. Save complited 
